# Introduction

Let us consider this example (array written in general format):

In [67]:
ls = [0, 1, 3, 6, 10]

Its following parts:

In [68]:
ls = [0, 1, 3, 6, 10]
ls = [1, 3, 6, 10]
ls = [3, 6, 10]
ls = [6, 10]
ls = [10]
ls = []

The corresponding sums are (put together in a list): `[20, 20, 19, 16, 10, 0]`

The function `parts_sums` (or its variants in other languages) will take as parameter a list ls and return a list of the sums of its parts as defined above.

In [69]:
ls = [1, 2, 3, 4, 5, 6] 
# --> parts_sums(ls) -> [21, 20, 18, 15, 11, 6, 0]

ls = [744125, 935, 407, 454, 430, 90, 144, 6710213, 889, 810, 2579358]
# --> parts_sums(ls) -> [10037855, 9293730, 9292795, 9292388, 9291934, 9291504, 9291414, 9291270, 2581057, 2580168, 2579358, 0]

**Notes**

- Take a look at performance: some lists have thousands of elements.

# Prerequisites

In [78]:
import random
from typing import List

In [158]:
N = 20_000
ls1 = []
ls1_result = [0]
ls2 = [0, 1, 3, 6, 10]
ls2_result = [20, 20, 19, 16, 10, 0]
ls3 = [1, 2, 3, 4, 5, 6]
ls3_result = [21, 20, 18, 15, 11, 6, 0]
ls4 = [744125, 935, 407, 454, 430, 90, 144, 6710213, 889, 810, 2579358]
ls4_result = [10037855, 9293730, 9292795, 9292388, 9291934, 9291504, 9291414, 9291270, 2581057, 2580168, 2579358, 0]
numbers = [random.randint(0, 10_000) for _ in range(N)]

# Native Python

## 'Naive' attempt

In [76]:
def sum_parts(ls: List[int]) -> List[int]:
    sums = []
    for i in range(len(ls)):
        sums.append(sum(ls[i:]))
    return sums + [0]

## List-Comp attempt

In [91]:
def sum_list_comp(ls: List[int]) -> List[int]:
    sums = [sum(ls[i:]) for i in range(len(ls))] + [0]
    return sums

# Smarter attempt

In [97]:
def difference(ls: List[int]) -> List[int]:
    total = sum(ls)
    sums = []
    sums.append(total)
    for n in ls:
        total -= n
        sums.append(total)
    return sums

# Even smarter attempt

In [116]:
def acc(ls: List[int]) -> List[int]:
    total = sum(ls)
    return [total] + [total - el for el in itertools.accumulate(ls)]

## Test Cases

### Unit Tests

In [92]:
ls = range(1, 7)

In [93]:
sum_parts(ls)

[21, 20, 18, 15, 11, 6, 0]

In [95]:
sum_list_comp(ls)

[21, 20, 18, 15, 11, 6, 0]

In [98]:
difference(ls)

[21, 20, 18, 15, 11, 6, 0]

In [156]:
assert(acc([]) == [0])
assert(acc(ls) == [21, 20, 18, 15, 11, 6, 0])
assert(acc([0, 1, 3, 6, 10]) == [20, 20, 19, 16, 10, 0])

### Benchmark Test

In [84]:
%%timeit -r 2 -n 1
sum_parts(numbers)

3.37 s Â± 6.6 ms per loop (mean Â± std. dev. of 2 runs, 1 loop each)


In [96]:
%%timeit -r 2 -n 1
sum_list_comp(numbers)

3.36 s Â± 1.37 ms per loop (mean Â± std. dev. of 2 runs, 1 loop each)


In [113]:
%%timeit
difference(numbers)

2.74 ms Â± 69.7 Âµs per loop (mean Â± std. dev. of 7 runs, 100 loops each)


In [115]:
%%timeit
acc(numbers)

1.97 ms Â± 12.7 Âµs per loop (mean Â± std. dev. of 7 runs, 1000 loops each)


# Numpy

Thanks to the preliminary work in the previous part, the following solutions are feasible.

In [122]:
import numpy as np

In [145]:
def sum_diff_np(ls: np.ndarray) -> np.ndarray:
    total = sum(ls)
    return np.insert(total - np.array([*itertools.accumulate(ls)]), 0, total)

## Test cases

## Unit Tests

In [123]:
ls = np.arange(1, 7)

In [150]:
assert(all(sum_diff_np(ls) == [21, 20, 18, 15, 11, 6, 0]))

In [152]:
assert(all(sum_diff_np(np.array([0, 1, 3, 6, 10])) == [20, 20, 19, 16, 10, 0]))

In [144]:
# keyword initial is new in Python 3.8
#[total - el for el in itertools.accumulate(ls, initial=0)]

array([21, 20, 18, 15, 11,  6,  0])

### Benchmark Test

# Numba Cuda

In [1]:
from numba import cuda
import numpy as np

In [46]:
@cuda.jit
def pp(in_array, out):
    i = cuda.grid(1)
    if i < in_array.size:
        for idx in range(i, in_array.size):
            out[i] += in_array[idx]

In [63]:
number_of_scenarios = 1_000_000
threads_per_block = 64  # number of threads within a blocks x dimension
blocks_per_grid = (number_of_scenarios + (threads_per_block - 1)) // threads_per_block  # blocks x dimension
in_array = np.random.randint(0, 10_000, number_of_scenarios)
out = np.zeros(number_of_scenarios).astype(np.int32)

In [66]:
%%timeit -r 7 -n 1
pp[blocks_per_grid, threads_per_block](in_array, out)

1.07 s Â± 5.16 ms per loop (mean Â± std. dev. of 7 runs, 1 loop each)


In [65]:
len(out)

1000000